In [1]:
import pandas as pd

df = pd.read_excel('D:\Liberty (Base data work)\Actaul Data with Reasons (S, N, W & E).xlsx')

# Step 1: Sort by CustomerID and Year (latest year first)
df = df.sort_values(by=['CustomerID', 'Start Year'], ascending=[True, False])

# Step 2: Get the latest churn label for each customer
df['Overall Churned'] = df.groupby('CustomerID')['Churn Label'].transform('first')

df.to_csv('Actaul Data with Reasons and Overall Churned.csv', index = False)

<>:3: SyntaxWarning: invalid escape sequence '\L'
<>:3: SyntaxWarning: invalid escape sequence '\L'
C:\Users\Admin\AppData\Local\Temp\ipykernel_3896\4115491648.py:3: SyntaxWarning: invalid escape sequence '\L'
  df = pd.read_excel('D:\Liberty (Base data work)\Actaul Data with Reasons (S, N, W & E).xlsx')


In [2]:
data = pd.read_csv('Actaul Data with Reasons and Overall Churned.csv')

In [3]:
import pandas as pd

# Calculate Total Revenue (Sum of Total Premium Payable for each customer)
data['total_revenue'] = data['Total Premium Payable (Overall)']

# Calculate total revenue for each customer
customer_total_revenue = data.groupby('CustomerID')['total_revenue'].sum()

# Calculate total number of purchases (policies) per customer
customer_total_purchases = data.groupby('CustomerID')['Number of Policies'].sum()

# Calculate Average Purchase Value (APV) for each customer
customer_apv = customer_total_revenue / customer_total_purchases

# Filter one record per customer (if needed for customer-level calculations)
data_customer_level = data.drop_duplicates(subset='CustomerID', keep='first')

# Calculate customer-specific Average Purchase Frequency (APF)
# APF is the total number of purchases divided by the number of unique customers
unique_customers = data_customer_level['CustomerID'].nunique()
customer_apf = customer_total_purchases / unique_customers

# Convert 'Overall Churned' column to binary for churn rate calculation
data_customer_level['Churned_Binary'] = data_customer_level['Overall Churned'].apply(lambda x: 1 if x == 'Yes' else 0)

# Calculate churn rate
churned_customers = data_customer_level[data_customer_level['Churned_Binary'] == 1]['CustomerID'].nunique()
churn_rate = churned_customers / unique_customers

# Calculate Average Customer Lifespan (ACL)
average_customer_lifespan = 1 / churn_rate if churn_rate != 0 else float('inf')

# Calculate customer-wise CLV
customer_clv = customer_apv * customer_apf * average_customer_lifespan

# Create a DataFrame for customer-wise CLV and other metrics
customer_clv_df = pd.DataFrame({
    'CustomerID': customer_total_revenue.index,
    'Customer_APV': customer_apv.values,
    'Customer_APF': customer_apf.values,
    'Churn_Rate': churn_rate,
    'Average_Customer_Lifespan': average_customer_lifespan,
    'CLV': customer_clv.values
})

# Display the top rows of the CLV DataFrame
print(customer_clv_df.head())

# Export the results to a CSV file
customer_clv_df.to_csv('customer_clv_updated (base data).csv', index=False)

C:\Users\Admin\AppData\Local\Temp\ipykernel_3896\1158506886.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_customer_level['Churned_Binary'] = data_customer_level['Overall Churned'].apply(lambda x: 1 if x == 'Yes' else 0)


  CustomerID  Customer_APV  Customer_APF  Churn_Rate  \
0     C00001       11975.0      0.000002    0.799132   
1     C00002       16876.0      0.000002    0.799132   
2     C00003       12725.0      0.000002    0.799132   
3     C00004        9012.0      0.000002    0.799132   
4     C00005       30920.0      0.000002    0.799132   

   Average_Customer_Lifespan       CLV  
0                   1.251358  0.023299  
1                   1.251358  0.032835  
2                   1.251358  0.024758  
3                   1.251358  0.017534  
4                   1.251358  0.060159  


In [4]:
churned_customers

513969

In [5]:
unique_customers

643159